# Homework 02: Open & Axial Coding Walkthrough

Grounded-theory error analysis for the email summariser workshop. Use this notebook to link qualitative insights from the annotation tool back into the Analyze → Measure → Improve loop.

## 0. Prerequisites
- Run Notebook 00/01a to prepare either the filtered or synthetic email CSV.
- Use `tools/generate_email_traces.py` to create traces for the current Git commit (short SHA stored as the `run_id`). The optional cell below lets you run it in-place during a workshop demo.
- Launch `tools/email_annotation_app.py` to collect open coding notes and failure modes in the browser, then return here to analyse the DuckDB tables.

In [ ]:
from pathlib import Path
import os
import json
import pandas as pd
import duckdb
from IPython.display import display, Markdown
import ipywidgets as widgets
from dotenv import load_dotenv

load_dotenv()

DATA_DIR = Path("../data")
TRACE_ROOT = Path("../annotation/traces")
DUCKDB_PATH = DATA_DIR / "email_annotations.duckdb"

### 1. Choose Attributes for Generating Traces


In [ ]:
DEFAULT_PROMPT_PATH = Path("../prompts/email_summary_prompt.txt")
DEFAULT_MODEL = os.environ.get("PYDANTIC_AI_MODEL")

model_dropdown = widgets.Dropdown(
    options=[
        ("GPT-5 mini", "openai:gpt-5-mini"),
        ("GPT-5", "openai:gpt-5"),
        ("Claude 4.5 Sonnet", "anthropic:claude-4-5-sonnet"),
    ],
    value=DEFAULT_MODEL,
    description="GenAI Model:",
    style={"description_width": "initial"},
)

prompt_path_widget = widgets.Text(
    value=str(DEFAULT_PROMPT_PATH.resolve()),
    description="Prompt Template:",
    style={"description_width": "initial"},
)

workers_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=30,
    step=1,
    description="LLM Workers:",
    style={"description_width": "initial"},
)

limit_slider = widgets.IntSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description="Email Limit:",
    style={"description_width": "initial"},
)

display(model_dropdown, prompt_path_widget, workers_slider, limit_slider)

In [ ]:
DATA_SOURCE_PATH = DATA_DIR / "curated_emails.csv"
Markdown(f"**Using:** `{DATA_SOURCE_PATH}`")

### 2. (Optional) Regenerate traces for this run
Set the flag to `True` to call `tools/generate_email_traces.py` using the selected dataset. The script writes trace JSON under `annotation/traces/<run_id>` and upserts rows into DuckDB.

In [ ]:
RUN_TRACE_GENERATOR = True  # toggle to True for live demos

if RUN_TRACE_GENERATOR:
    import subprocess
    import sys

    cmd = [
        sys.executable,
        str(Path("../tools/generate_email_traces.py")),
        "--emails",
        str(DATA_SOURCE_PATH.resolve()),
        "--out",
        str(TRACE_ROOT.resolve()),
        "--model",
        model_dropdown.value,
        "--prompt",
        str(Path(prompt_path_widget.value).resolve()),
        "--workers",
        str(workers_slider.value),
        "--limit",
        str(limit_slider.value),
    ]
    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True)

### 3. Verify available trace runs
Each trace directory name equals the short Git SHA captured when the generator script ran.

In [ ]:
available_runs = []
if TRACE_ROOT.exists():
    available_runs = sorted(p.name for p in TRACE_ROOT.iterdir() if p.is_dir())

if not available_runs:
    raise RuntimeError(
        "No trace runs detected under ../annotation/traces. Generate traces before proceeding."
    )

# Connect to DB to get the latest run by generated_at timestamp
temp_conn = duckdb.connect(str(DUCKDB_PATH), read_only=True)
latest_run = temp_conn.execute("SELECT run_id FROM trace_runs ORDER BY generated_at DESC LIMIT 1").fetchone()
temp_conn.close()

if latest_run:
    ACTIVE_RUN_ID = latest_run[0]
else:
    # Fallback to alphabetically last directory if no runs in DB
    ACTIVE_RUN_ID = available_runs[-1]

Markdown(f"**Active run:** `{ACTIVE_RUN_ID}` (latest by timestamp)")

### Annotation Tool (Browser UI)
- **Launch:** `uv run python tools/email_annotation_app.py` from the repo root.
- **Navigate:** open `http://localhost:5001`.
- **Controls:** 
  - `Z` – mark pass (instant judgment)
  - `X` – mark fail (instant judgment)
  - `Esc` – remove judgment (instant delete)
  - `N` – add/edit note (observation)
  - `F` – link failure modes (fail only)
  - `←/→` – navigate between emails
- **Auto-save:** every change is written directly to DuckDB.
- Capture judgments, notes, and failure modes before returning to this notebook to analyze results.

### 4. Connect to DuckDB

In [ ]:
conn = duckdb.connect(str(DUCKDB_PATH))

## Schema Exploration
Explore each table in the DuckDB annotation database to understand the data model.

### 1. `labelers` table
**Purpose**: Tracks who is annotating emails in the workshop.

In [ ]:
labelers_df = conn.execute("SELECT * FROM labelers ORDER BY created_at").df()
display(Markdown(f"**Columns**: `{', '.join(labelers_df.columns.tolist())}`"))
display(labelers_df)

### 2. `trace_runs` table
**Purpose**: Tracks each trace generation run (identified by Git commit SHA).

In [ ]:
runs_df = conn.execute("SELECT * FROM trace_runs ORDER BY generated_at DESC").df()
display(Markdown(f"**Columns**: `{', '.join(runs_df.columns.tolist())}`"))
display(runs_df)

### 3. `email_judgments` table
**Purpose**: Stores pass/fail decisions for each email. One row per email/labeler/run.

In [ ]:
ACTIVE_RUN_ID

In [ ]:
judgments_df = conn.execute(
    "SELECT * FROM email_judgments WHERE run_id = ? ORDER BY judged_at DESC LIMIT 10",
    (ACTIVE_RUN_ID,)
).df()
display(Markdown(f"**Columns**: `{', '.join(judgments_df.columns.tolist())}`"))
display(judgments_df)

### 4. `annotations` table
**Purpose**: Stores optional observation notes for emails (one note per email/labeler).

In [ ]:
annotations_df = conn.execute(
    "SELECT * FROM annotations WHERE run_id = ? ORDER BY created_at DESC LIMIT 10",
    (ACTIVE_RUN_ID,)
).df()
display(Markdown(f"**Columns**: `{', '.join(annotations_df.columns.tolist())}`"))
display(annotations_df)

### 5. `failure_modes` table
**Purpose**: Catalog of failure mode definitions used for axial coding.

In [ ]:
failure_modes_df = conn.execute("SELECT * FROM failure_modes ORDER BY display_name").df()
display(Markdown(f"**Columns**: `{', '.join(failure_modes_df.columns.tolist())}`"))
display(failure_modes_df)

### 6. `axial_links` table
**Purpose**: Links annotations to failure modes (many-to-many relationship).

In [ ]:
axial_links_df = conn.execute(
    "SELECT * FROM axial_links WHERE run_id = ? ORDER BY linked_at DESC LIMIT 10",
    (ACTIVE_RUN_ID,)
).df()
display(Markdown(f"**Columns**: `{', '.join(axial_links_df.columns.tolist())}`"))
display(axial_links_df)

## Denormalized Master Table
Create a comprehensive view joining: email content + metadata + judgments + notes + failure modes.

In [ ]:
def load_emails_from_traces(run_id: str) -> pd.DataFrame:
    """Load all emails for a run from trace JSON files."""
    run_dir = TRACE_ROOT / run_id
    if not run_dir.exists():
        return pd.DataFrame()
    
    emails = []
    for trace_file in sorted(run_dir.glob("trace_*.json")):
        email_hash = trace_file.stem.replace("trace_", "")
        try:
            trace_data = json.loads(trace_file.read_text(encoding="utf-8"))
            metadata = trace_data.get("metadata", {}).get("extra", {})
            
            # Extract subject from metadata
            subject = metadata.get("normalized_subject", "").title() or "(no subject)"
            
            # Extract body from request content
            request_content = trace_data.get("request", {}).get("messages", [{}])[0].get("content", "")
            body_match = request_content.split("```")
            body = body_match[1].strip() if len(body_match) > 2 else ""
            
            emails.append({
                "email_hash": email_hash,
                "subject": subject,
                "body": body,
                "Intent": metadata.get("Intent", None),
                "Designation": metadata.get("Designation", None),
                "Tone": metadata.get("Tone", None),
                "Context": metadata.get("Context", None),
                "run_id": run_id,
            })
        except (json.JSONDecodeError, IOError):
            continue
    
    return pd.DataFrame(emails)

# Load all emails from trace files
master_df = load_emails_from_traces(ACTIVE_RUN_ID)
display(Markdown(f"Loaded **{len(master_df):,}** emails from trace files"))

In [ ]:
# Get all judgments for this run
all_judgments = conn.execute(
    """
    SELECT email_hash, labeler_id, pass_fail, judged_at
    FROM email_judgments
    WHERE run_id = ?
    """,
    (ACTIVE_RUN_ID,)
).df()

# Get all annotations for this run
all_annotations = conn.execute(
    """
    SELECT email_hash, labeler_id, open_code
    FROM annotations
    WHERE run_id = ?
    """,
    (ACTIVE_RUN_ID,)
).df()

# Get all failure modes linked to emails in this run
failure_mode_links = conn.execute(
    """
    SELECT 
        a.email_hash,
        fm.display_name,
        fm.definition
    FROM axial_links al
    JOIN annotations a ON al.annotation_id = a.annotation_id
    JOIN failure_modes fm ON al.failure_mode_id = fm.failure_mode_id
    WHERE al.run_id = ?
    """,
    (ACTIVE_RUN_ID,)
).df()

# Aggregate failure modes per email (comma-separated)
if not failure_mode_links.empty:
    failure_agg = failure_mode_links.groupby("email_hash").agg({
        "display_name": lambda x: ", ".join(sorted(set(x))),
        "definition": lambda x: " | ".join(sorted(set(x)))
    }).reset_index()
    failure_agg.columns = ["email_hash", "failure_modes", "failure_mode_definitions"]
else:
    failure_agg = pd.DataFrame(columns=["email_hash", "failure_modes", "failure_mode_definitions"])

display(Markdown(f"Found **{len(all_judgments):,}** judgments, **{len(all_annotations):,}** notes, **{len(failure_mode_links):,}** failure mode links"))

In [ ]:
# Join everything together (left join so unjudged emails show NULL)
master_df = master_df.merge(
    all_judgments[["email_hash", "pass_fail", "judged_at", "labeler_id"]], 
    on="email_hash", 
    how="left"
)
master_df = master_df.merge(
    all_annotations[["email_hash", "open_code"]], 
    on="email_hash", 
    how="left"
)
master_df = master_df.merge(
    failure_agg, 
    on="email_hash", 
    how="left"
)

# Reorder columns for readability (only include columns that exist)
base_columns = ["email_hash", "subject", "body"]
metadata_columns = ["Intent", "Designation", "Tone", "Context"]
judgment_columns = ["pass_fail", "judged_at", "labeler_id", "open_code", "failure_modes", "failure_mode_definitions", "run_id"]

# Build column order: base + existing metadata + judgments
column_order = base_columns + [col for col in metadata_columns if col in master_df.columns] + judgment_columns
column_order = [col for col in column_order if col in master_df.columns]

master_df = master_df[column_order]

display(Markdown(f"### Master Table: {len(master_df):,} rows"))
display(Markdown("One row per email. Unjudged emails show NULL for judgment columns."))
master_df.head(10)

In [ ]:
master_df[~master_df['failure_modes'].isnull()]

## Export Master Table
Export the denormalized table as CSV for review in Excel/Google Sheets.

In [ ]:
EXPORT = True  # Toggle to False to skip export

if EXPORT:
    export_path = DATA_DIR / f"email_analysis_{ACTIVE_RUN_ID}.csv"
    master_df.to_csv(export_path, index=False)
    display(Markdown(f"✅ **Exported** → `{export_path.name}`"))
    display(Markdown(f"- **Total emails**: {len(master_df):,}"))
    display(Markdown(f"- **Judged**: {master_df['pass_fail'].notna().sum():,}"))
    display(Markdown(f"- **Pass**: {(master_df['pass_fail'] == True).sum():,}"))
    display(Markdown(f"- **Fail**: {(master_df['pass_fail'] == False).sum():,}"))
    display(Markdown(f"- **Unjudged**: {master_df['pass_fail'].isna().sum():,}"))
else:
    display(Markdown("⚠️ Export disabled. Set `EXPORT = True` to write CSV."))